In [ ]:
from htmd import *
htmd.config(viewer='ngl')
# %qtconsole
#os.chdir('/webdata/73hboiwia98hdj209jq0/')  # Skip this command.

# System building: trypsin-benzamidine

Courtesy of [Stefan Doerr](https://github.com/stefdoerr).

If you don't have the `bentryp` directory, download the required files for the tutorial from this [link](http://pub.htmd.org/73hboiwia98hdj209jq0/bentryp.tar.gz). 

## Obtain structures

In [ ]:
prot = Molecule('3PTB')

## Clean structures

Water molecules and one calcium ion present in the PDB crystal structure were also read by HTMD. Start by removing the ligand:

In [ ]:
prot.remove('resname BEN')

## Preparing the protein

In [ ]:
prot = proteinPrepare(prot)

## Define segments

To build a system in HTMD, we need to separate the chemical molecules into separate segments. This prevents the builder from accidentally bonding different chemical molecules and allows us to add caps to them.

In [ ]:
prot.set('segid', 'P', sel='protein')
prot.set('segid', 'W', sel='water')
prot.set('segid', 'CA', sel='resname CA')

Center the protein to the origin

In [ ]:
prot.center()

## Let's add a ligand!

In [ ]:
ligand = Molecule('bentryp/benzamidine.pdb')
ligand.center()
ligand.view(sel='resname MOL', style='Licorice')

But the ligand is located inside the protein...
We would like the ligand to be:

* At a certain distance from the protein
* Rotated randomly, to provide different starting conditions

## Let's reposition the ligand then

In [ ]:
ligand.rotateBy(uniformRandomRotation())

This took care of the ligand rotation. But now we still need to position it far from the protein.
We need to find out the radius of the protein:

![](http://pub.htmd.org/73hboiwia98hdj209jq0/maxdist.png)

In [ ]:
from htmd.molecule.util import maxDistance
D = maxDistance(prot, 'all')
print(D)

In [ ]:
D += 10
ligand.moveBy([D, 0, 0])  # Move the ligand 10 Angstrom away from the furthest protein atom in X dimension
ligand.rotateBy(uniformRandomRotation())

## Don't forget to add a segment

In [ ]:
ligand.set('segid','L')
ligand.set('resname','MOL')

### Mix it all together

In [ ]:
mol = Molecule(name='combo')
mol.append(prot)
mol.append(ligand)
mol.reps.add(sel='protein', style='NewCartoon', color='Secondary Structure')
mol.reps.add(sel='resname MOL', style='Licorice')
mol.view()

## Solvate

> Water is the driving force of all nature.
>
> -Leonardo da Vinci

![](http://pub.htmd.org/73hboiwia98hdj209jq0/waterbox.png)

In [ ]:
D = D + 5
smol = solvate(mol, minmax=[[-D, -D, -D], [D, D, D]])
smol.reps.add(sel='water', style='Lines')
smol.view()

## Available CHARMM forcefields

In [ ]:
charmm.listFiles()

## Build and ionize

In [ ]:
topos  = ['top/top_all22star_prot.rtf', 'bentryp/benzamidine.rtf']
params = ['par/par_all22star_prot.prm', 'bentryp/benzamidine.prm']

bmol = charmm.build(smol, topo=topos, param=params, 
                        outdir='03b_out_build')

In [ ]:
bmol.view(sel='protein', style='NewCartoon', 
          color='Secondary Structure', hold=True)
bmol.view(sel='resname MOL', style='Licorice', hold=True)
bmol.view(sel='ions', style='VDW', hold=True)
bmol.view(sel='water', style='Lines')

In [ ]:
bmol